In [1]:
import time
import os
from Research.backtest.bt import BTDaily
import matplotlib.pyplot as plt
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
import AlphaNet
from AlphaNet.Data import DataLoader
from AlphaNet.Models import AlphaNet_LSTM_V1
# read_task
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


In [5]:
task_info["Factor"]

,Alpha_Name,shift,sequence,target,LR,epoch_num,alpha_list,universe,status
factor_id,,,,,,,,,
0,AlphaNet_ConcatMoving5And10,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_5, COV_open_high_10, COV_open_l...",Investable,finished
1,AlphaNet_No_Decay,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",Investable,finished
2,AlphaNet_No_Return,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",Investable,finished
3,AlphaNet_No_ZSCORE,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",Investable,finished
4,AlphaNet_No_STD,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",Investable,finished
5,AlphaNet_No_CORR,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",Investable,finished
6,AlphaNet_No_COV,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[CORR_open_high_10, CORR_open_low_10, CORR_ope...",Investable,finished
7,AlphaNetV1,1,20,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",Investable,finished
8,AlphaNet_8Input,1,20,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_div_open_close_div_high_close_10, COV_div...",Investable,finished


In [7]:
alpha_name = "AlphaNetV4_Moving10"
shift = 1
sequence = 20
target = "10dRet"
LR = [0.001,0.00001,0.0000001]
epoch_num = [30,30,30]
universe = "Investable"

In [6]:
alpha_list = pd.read_parquet("/home/wuwenjun/Data/AlphaNetV4_Moving10/2016/20160118.par")
alpha_list = list(alpha_list.columns)
alpha_list

['COV_div_open_close_div_high_close_10',
 'COV_div_open_close_div_low_close_10',
 'COV_div_open_close_close_10',
 'COV_div_open_close_volume_sqrt_8_10',
 'COV_div_open_close_pct_chg_10',
 'COV_div_open_close_div_close_free_turnover_10',
 'COV_div_open_close_div_low_high_10',
 'COV_div_high_close_div_low_close_10',
 'COV_div_high_close_close_10',
 'COV_div_high_close_volume_sqrt_8_10',
 'COV_div_high_close_pct_chg_10',
 'COV_div_high_close_div_close_free_turnover_10',
 'COV_div_high_close_div_low_high_10',
 'COV_div_low_close_close_10',
 'COV_div_low_close_volume_sqrt_8_10',
 'COV_div_low_close_div_close_free_turnover_10',
 'COV_div_low_close_div_low_high_10',
 'COV_close_div_vwap_close_10',
 'COV_close_volume_sqrt_8_10',
 'COV_close_pct_chg_10',
 'COV_close_div_close_free_turnover_10',
 'COV_close_div_low_high_10',
 'COV_div_vwap_close_volume_sqrt_8_10',
 'COV_div_vwap_close_div_close_free_turnover_10',
 'COV_div_vwap_close_div_low_high_10',
 'COV_div_vwap_close_div_vwap_close_2_10',
 

In [4]:
alpha_list = [
# 'open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover',
'div_open_close','div_high_close','div_low_close','close','div_vwap_close','volume_sqrt','pct_chg','turnover'
#             'open_ts_norm_30d',
#              'high_ts_norm_30d',
#              'low_ts_norm_30d',
#              'close_ts_norm_30d',
#              'vwap_ts_norm_30d',
#              'volume_ts_norm_30d',
#              'pct_chg_ts_norm_30d',
#              'turnover_ts_norm_30d',
#              'free_turnover_ts_norm_30d'
             ]
feat_list = alpha_list.copy()
method = ["CORR","STD","ZSCORE","RETURN","DECAY","COV"]
from AlphaNet.Data import generate_alpha_list
alpha_list = generate_alpha_list(feat_list,method,day=10)
alpha_list
# alpha_list = [i +"_residual" for i in feat_list]

['COV_div_open_close_div_high_close_10',
 'COV_div_open_close_div_low_close_10',
 'COV_div_open_close_close_10',
 'COV_div_open_close_div_vwap_close_10',
 'COV_div_open_close_volume_sqrt_10',
 'COV_div_open_close_pct_chg_10',
 'COV_div_open_close_turnover_10',
 'COV_div_high_close_div_low_close_10',
 'COV_div_high_close_close_10',
 'COV_div_high_close_div_vwap_close_10',
 'COV_div_high_close_volume_sqrt_10',
 'COV_div_high_close_pct_chg_10',
 'COV_div_high_close_turnover_10',
 'COV_div_low_close_close_10',
 'COV_div_low_close_div_vwap_close_10',
 'COV_div_low_close_volume_sqrt_10',
 'COV_div_low_close_pct_chg_10',
 'COV_div_low_close_turnover_10',
 'COV_close_div_vwap_close_10',
 'COV_close_volume_sqrt_10',
 'COV_close_pct_chg_10',
 'COV_close_turnover_10',
 'COV_div_vwap_close_volume_sqrt_10',
 'COV_div_vwap_close_pct_chg_10',
 'COV_div_vwap_close_turnover_10',
 'COV_volume_sqrt_pct_chg_10',
 'COV_volume_sqrt_turnover_10',
 'COV_pct_chg_turnover_10',
 'CORR_div_open_close_div_high_clo

In [8]:
factor_df = pd.DataFrame([alpha_name,shift,sequence,target,LR,epoch_num,alpha_list,universe],index=[
    'Alpha_Name','shift', 'sequence','target', 'LR', 'epoch_num' ,'alpha_list',"universe"
]).T
factor_df.index.names = ["factor_id"]
factor_df["status"] = "waiting"

In [9]:
task_info["Factor"] = task_info["Factor"].append(factor_df)
task_info["Factor"].reset_index(inplace=True,drop=True)
task_info["Factor"].index.names = ["factor_id"]
task_info["Factor"]

,Alpha_Name,shift,sequence,target,LR,epoch_num,alpha_list,universe,status
factor_id,,,,,,,,,
0,AlphaNet_ConcatMoving5And10,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_5, COV_open_high_10, COV_open_l...",Investable,finished
1,AlphaNet_No_Decay,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",Investable,finished
2,AlphaNet_No_Return,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",Investable,finished
3,AlphaNet_No_ZSCORE,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",Investable,finished
4,AlphaNet_No_STD,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",Investable,finished
5,AlphaNet_No_CORR,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",Investable,finished
6,AlphaNet_No_COV,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[CORR_open_high_10, CORR_open_low_10, CORR_ope...",Investable,finished
7,AlphaNetV1,1,20,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",Investable,finished
8,AlphaNet_8Input,1,20,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_div_open_close_div_high_close_10, COV_div...",Investable,finished


In [10]:
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/Task.npy",task_info)